
# Sérialisation

Le notebook explore différentes façons de sérialiser des données et leurs limites.

## JSON

Le format :epkg:`JSON` est le format le plus utilisé sur internet
notemmant via les :epkg:`API REST`.

### Ecriture (json)


In [ ]:
from io import StringIO, BytesIO
import timeit
import json
import numpy
import ujson
import cloudpickle
import pickle
import matplotlib.pyplot as plt
import pandas


data = {
    "records": [
        {
            "nom": "Xavier",
            "prénom": "Xavier",
            "langages": [{"nom": "C++", "age": 40}, {"nom": "Python", "age": 20}],
        }
    ]
}

In [ ]:
buffer = StringIO()
res = json.dump(data, buffer)  # 1
seq = buffer.getvalue()
seq

### Lecture (json)



In [ ]:
buffer = StringIO(seq)
read = json.load(buffer)
read

### Limite

Les matrices :epkg:`numpy` ne sont pas sérialisables facilement.



In [ ]:
data = {"mat": numpy.array([0, 1])}

buffer = StringIO()
try:
    json.dump(data, buffer)
except Exception as e:
    print(e)

Les classes ne sont pas sérialisables non plus facilement.



In [ ]:
class A:
    def __init__(self, att):
        self.att = att


data = A("e")
buffer = StringIO()
try:
    json.dump(data, buffer)
except Exception as e:
    print(e)

Pour ce faire, il faut indiquer au module :mod:`json`
comment convertir la classe en un ensemble de listes et dictionnaires et
la classe :class:`json.JSONEncoder`.



In [ ]:
class MyEncoder(json.JSONEncoder):
    def default(self, o):
        return {"classname": o.__class__.__name__, "data": o.__dict__}


data = A("e")
buffer = StringIO()
res = json.dump(data, buffer, cls=MyEncoder)
res = buffer.getvalue()
res

Et la relecture avec la classe :class:`json.JSONDecoder`.



In [ ]:
class MyDecoder(json.JSONDecoder):
    def decode(self, o):
        dec = json.JSONDecoder.decode(self, o)
        if isinstance(dec, dict) and dec.get("classname") == "A":
            return A(dec["data"]["att"])
        else:
            return dec


buffer = StringIO(res)
obj = json.load(buffer, cls=MyDecoder)
obj

### Sérialisation rapide

Le module :mod:`json` est la librairie standard de Python mais comme
la sérialisation au format *JSON* est un besoin très fréquent,
il existe des alternative plus rapide comme :epkg:`ujson`.



In [ ]:
data = {
    "records": [
        {
            "nom": "Xavier",
            "prénom": "Xavier",
            "langages": [{"nom": "C++", "age": 40}, {"nom": "Python", "age": 20}],
        }
    ]
}

In [ ]:
data_time = []
expression = "json.dump(data, StringIO())"
d = timeit.timeit(expression, globals=globals(), number=100)
data_time.append(dict(expression=expression, time=d))
d

In [ ]:
expression = "ujson.dump(data, StringIO())"
d = timeit.timeit(expression, globals=globals(), number=100)
data_time.append(dict(expression=expression, time=d))
d

Ces deux lignes mesures l'écriture au format JSON
mais il faut aussi mesurer la lecture.



In [ ]:
buffer = StringIO()
ujson.dump(data, buffer)
res = buffer.getvalue()

expression = "json.load(StringIO(res))"
d = timeit.timeit(expression, globals=globals(), number=100)
data_time.append(dict(expression=expression, time=d))
d

In [ ]:
expression = "ujson.load(StringIO(res))"
d = timeit.timeit(expression, globals=globals(), number=100)
data_time.append(dict(expression=expression, time=d))
d

On enlève le temps passé dans la creation du buffer.



In [ ]:
expression = "StringIO(res)"
d = timeit.timeit(expression, globals=globals(), number=100)
data_time.append(dict(expression=expression, time=d))
d

## Pickle

Le module :mod:`pickle` effectue la même chose mais au format binaire.
Celui-ci est propre à *Python* et ne peut être lu d'autres langages,
voire parfois par d'autres versions de *Python*.

### Ecriture (pickle)



In [ ]:
data = {
    "records": [
        {
            "nom": "Xavier",
            "prénom": "Xavier",
            "langages": [{"nom": "C++", "age": 40}, {"nom": "Python", "age": 20}],
        }
    ]
}

In [ ]:
buffer = BytesIO()
res = pickle.dump(data, buffer)
seq = buffer.getvalue()
seq

### Lecture (pickle)



In [ ]:
buffer = BytesIO(seq)
read = pickle.load(buffer)
read

### Les classes

A l'inverse du format *JSON*, les classes sont sérialisables avec
:mod:`pickle` parce que le langage utilise un format très proche
de ce qu'il a en mémoire. Il n'a pas besoin de conversion supplémentaire.



In [ ]:
data = A("r")
buffer = BytesIO()
res = pickle.dump(data, buffer)
seq = buffer.getvalue()
seq

In [ ]:
buffer = BytesIO(seq)
read = pickle.load(buffer)
read

### Réduire la taille

Certaines informations sont duppliquées et il est préférable de ne pas
les sérialiser deux fois surtout si elles sont voluminueuses.



In [ ]:
class B:
    def __init__(self, att):
        self.att1 = att
        self.att2 = att

In [ ]:
data = B("r")
buffer = BytesIO()
res = pickle.dump(data, buffer)
seq = buffer.getvalue()
seq

Evitons maintenant de stocker deux fois le même attribut.



In [ ]:
class B:
    def __init__(self, att):
        self.att1 = att
        self.att2 = att

    def __getstate__(self):
        return dict(att=self.att1)


data = B("r")
buffer = BytesIO()
res = pickle.dump(data, buffer)
seq = buffer.getvalue()
seq

C'est plus court mais il faut inclure maintenant la relecture.



In [ ]:
class B:
    def __init__(self, att):
        self.att1 = att
        self.att2 = att

    def __getstate__(self):
        return dict(att=self.att1)

    def __setstate__(self, state):
        setattr(self, "att1", state["att"])
        setattr(self, "att2", state["att"])


buffer = BytesIO(seq)
read = pickle.load(buffer)
read

In [ ]:
read.att1, read.att2

In [ ]:
data = B("r")
expression = "pickle.dump(data, BytesIO())"
d = timeit.timeit(expression, globals=globals(), number=100)
data_time.append(dict(expression=expression, time=d))
d

In [ ]:
expression = "pickle.load(BytesIO(seq))"
d = timeit.timeit(expression, globals=globals(), number=100)
data_time.append(dict(expression=expression, time=d))
d

La sérialisation binaire est habituellement plus rapide dans les langages
bas niveau comme C++. La même comparaison pour un langage haut niveau
tel que Python n'est pas toujours prévisible.
Il est possible d'accélérer un peu les choses.



In [ ]:
expression = "pickle.dump(data, BytesIO(), protocol=pickle.HIGHEST_PROTOCOL)"
d = timeit.timeit(expression, globals=globals(), number=100)
data_time.append(dict(expression=expression, time=d))
d

## Cas des fonctions

La sérialisation s'applique à des données et non à du code mais le
fait de sérialiser des fonctions est tout de même tentant.
La sérialisation binaire fonctionne même avec les fonctions.

### Binaire



In [ ]:
def myfunc(x):
    return x + 1


data = {"x": 5, "f": myfunc}


buffer = BytesIO()
res = pickle.dump(data, buffer)
buffer.getvalue()

In [ ]:
res = pickle.load(BytesIO(buffer.getvalue()))
res

In [ ]:
res["f"](res["x"])

La sérialisation ne conserve pas le code de la fonction, juste son nom.
Cela veut dire que si elle n'est pas disponible lorsqu'elle est appelée,
il sera impossible de s'en servir.



In [ ]:
del myfunc


try:
    pickle.load(BytesIO(buffer.getvalue()))
except Exception as e:
    print(e)

Il est possible de contourner l'obstacle en utilisant le module
:epkg:`cloudpickle` qui stocke le code de la fonction.



In [ ]:
def myfunc(x):
    return x + 1


data = {"x": 5, "f": myfunc}


buffer = BytesIO()
res = cloudpickle.dump(data, buffer)
buffer.getvalue()

In [ ]:
del myfunc


res = cloudpickle.load(BytesIO(buffer.getvalue()))
res

In [ ]:
res["f"](res["x"])

### Fonction et JSON

La sérialisation d'une fonction au format JSON ne
fonctionne pas avec le module standard.



In [ ]:
buffer = StringIO()
try:
    json.dump(data, buffer)  # 2
except Exception as e:
    print(e)

La sérialisation avec :epkg:`ujson` ne fonctionne pas non plus
même si elle ne produit pas toujours d'erreur.



In [ ]:
buffer = StringIO()
try:
    res = ujson.dump(data, buffer)  # 3
except TypeError as e:
    print(e)
buffer.getvalue()

## Cas des itérateurs

Les itérateurs fonctionnent avec la sérialisation binaire mais ceci
implique de stocker l'ensemble que l'itérateur parcourt.



In [ ]:
ens = [1, 2]

data = {"x": 5, "it": iter(ens)}


buffer = BytesIO()
res = pickle.dump(data, buffer)  # 4
buffer.getvalue()

In [ ]:
del ens

res = pickle.load(BytesIO(buffer.getvalue()))
res

In [ ]:
list(res["it"])

In [ ]:
list(res["it"])

## Cas des générateurs

Ils ne peuvent être sérialisés car le langage n'a pas accès à l'ensemble
des éléments que le générateur parcourt. Il n'y a aucun moyen de
sérialiser un générateur mais on peut sérialiser la fonction qui crée le générateur.



In [ ]:
def ensgen():
    yield 1
    yield 2


data = {"x": 5, "it": ensgen()}


buffer = BytesIO()
try:
    pickle.dump(data, buffer)
except Exception as e:
    print(e)

## Summary



In [ ]:
df = pandas.DataFrame(data_time)
print(df)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
df.set_index("expression").plot.barh(ax=ax[0])
df.loc[0, "time"] = numpy.nan
df.set_index("expression").plot.barh(ax=ax[1])
ax[0].set_title("Time")
ax[1].set_title("Time without `json.dump`")
fig.tight_layout()
fig.savefig("plot_serialisation_examples.png")